In [ ]:
import os
import glob
import pandas as pd

from embedding_functions import *

In [ ]:
# data import

data_path = './../data/raw/20news-bydate/20news-bydate-train'

# get list of category directories
categories = [d for d in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, d))]

# create lists to store texts and corresponding categories
texts = []
labels = []

# Loop through each category directory to read the text files and assign the appropriate label.
for category in categories:
    category_path = os.path.join(data_path, category)

    # import all text files in the current category folder
    file_paths = glob.glob(os.path.join(category_path, '*'))

    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
            text = file.read()
            texts.append(text)
            labels.append(category)

# create data frame
df = pd.DataFrame({'text': texts, 'category': labels})

In [ ]:
embeddings = []

for idx, text in enumerate(df['text']):
    try:
        embedding = get_embedding(text)
    except Exception as e:
        print(f"Error generating embedding for index {idx}: {e}")
    
    embeddings.append(embedding)

    # if necessary ... short delay to avoid API rate limits
    # time.sleep(0.1)

# add embeddings to data frame
df['embedding'] = embeddings

In [ ]:
# save data frame as pickle file in 'processed' folder
df.to_pickle('./../data/processed/training_data_with_embeddings.pkl')